

---

Templates For Front End :
* base
* index.
* admin .
* Event .
* Congratulations
* register .
* Login .
* Styling .
---


---





In [1]:
%%bash
mkdir -p templates static/{css,js,images}

In [2]:
%%writefile templates/base.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Event Booking System</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/styles.css') }}">
    <meta name="csrf-token" content="{{ csrf_token() }}">
</head>
<body>
    <header class="bg-blue-600 shadow-md">
        <nav class="container mx-auto px-4 py-3 flex justify-between items-center">
            <div class="logo text-2xl font-bold text-white">EventBook</div>
            <div class="nav-links flex items-center space-x-4 text-white">
                <a href="{{ url_for('home') }}" class="hover:underline">Home</a>
                {% if current_user %}
                    {% if current_user.role == 'admin' %}
                        <a href="{{ url_for('admin_panel') }}" class="hover:underline">Admin</a>
                    {% endif %}
                    <form action="{{ url_for('logout') }}" method="POST" style="display: inline;">
                        <input type="hidden" name="csrf_token" value="{{ csrf_token() }}"/>
                        <button type="submit" class="btn-link hover:underline">Logout</button>
                    </form>
                {% else %}
                    <a href="{{ url_for('login_page') }}" class="hover:underline">Login</a>
                    <a href="{{ url_for('register_page') }}" class="hover:underline">Register</a>
                {% endif %}
            </div>
        </nav>
    </header>

    <main class="container mx-auto px-4 py-6 min-h-screen">
        {% with messages = get_flashed_messages(with_categories=true) %}
            {% if messages %}
                <div class="flash-messages mb-6">
                    {% for category, message in messages %}
                        <div class="flash {{ category }} p-4 mb-2 rounded-md
                                    {% if category == 'success' %}bg-green-100 text-green-800
                                    {% elif category == 'error' %}bg-red-100 text-red-800
                                    {% else %}bg-blue-100 text-blue-800{% endif %}">
                            {{ message }}
                        </div>
                    {% endfor %}
                </div>
            {% endif %}
        {% endwith %}
        {% block content %}{% endblock %}
    </main>

    <footer class="bg-blue-600 text-white py-4">
        <div class="container mx-auto px-4 text-center">
            <p>&copy; 2025 Event Booking System</p>
        </div>
    </footer>
  <div id="toast-container" class="position-fixed bottom-0 end-0 p-3" style="z-index: 11"></div>
    <script>
    document.addEventListener('DOMContentLoaded', function() {
        // Get CSRF token
        function getCSRFToken() {
            return document.querySelector('meta[name="csrf-token"]').content;
        }

        // Handle delete buttons for events
        document.addEventListener('click', function(e) {
            // Event deletion
            if (e.target.classList.contains('btn-delete-event')) {
                if (!confirm('Are you sure you want to delete this event?')) return;

                const eventId = e.target.getAttribute('data-id');
                fetch(`/events/${eventId}`, {
                    method: 'DELETE',
                    headers: {
                        'Content-Type': 'application/json',
                        'X-CSRFToken': getCSRFToken()
                    }
                })
                .then(response => {
                    if (!response.ok) throw new Error('Failed to delete event');
                    return response.json();
                })
                .then(data => {
                    if (data.success) {
                        document.querySelector(`tr[data-id="${eventId}"]`).remove();
                    } else {
                        throw new Error(data.error || 'Unknown error');
                    }
                })
                .catch(error => {
                    console.error('Error:', error);
                    alert('Error deleting event: ' + error.message);
                });
            }

            // User deletion
            if (e.target.classList.contains('btn-delete-user')) {
                if (!confirm('Are you sure you want to delete this user?')) return;

                const userId = e.target.getAttribute('data-id');
                fetch(`/users/${userId}`, {
                    method: 'DELETE',
                    headers: {
                        'Content-Type': 'application/json',
                        'X-CSRFToken': getCSRFToken()
                    }
                })
                .then(response => {
                    if (!response.ok) throw new Error('Failed to delete user');
                    return response.json();
                })
                .then(data => {
                    if (data.success) {
                        document.querySelector(`tr[data-id="${userId}"]`).remove();
                    } else {
                        throw new Error(data.error || 'Unknown error');
                    }
                })
                .catch(error => {
                    console.error('Error:', error);
                    alert('Error deleting user: ' + error.message);
                });
            }
        });
    });
    </script>
</body>
</html>

Overwriting templates/base.html


In [3]:
%%writefile templates/index.html
{% extends "base.html" %}
{% block content %}
<div class="container">
    <h1>Upcoming Events</h1>
    <div class="events-grid">
        {% for event in events %}
        <div class="event-card">
            <img src="{{ event.image_url or 'https://via.placeholder.com/300' }}" alt="{{ event.name }}">
            <h3>{{ event.name }}</h3>
            <p>{{ event.date.strftime('%B %d, %Y at %I:%M %p') }}</p>
            <p>{{ event.venue }}</p>
            <p>${{ "%.2f"|format(event.price) }}</p>
            {% if event.id in booked_events %}
                <span class="booked-label">Booked</span>
            {% else %}
                <a href="/event/{{ event.id }}" class="btn">View Details</a>
            {% endif %}
        </div>
        {% endfor %}
    </div>
</div>
{% endblock %}

Overwriting templates/index.html


In [4]:
%%writefile templates/event.html
{% extends "base.html" %}
{% block content %}
<div class="container">
    <div class="event-details">
        <img src="{{ event.image_url or 'https://via.placeholder.com/600' }}" alt="{{ event.name }}" class="event-image">
        <div class="event-info">
            <h1>{{ event.name }}</h1>
            <p>{{ event.description }}</p>
            <div class="event-meta">
                <p><strong>Category:</strong> {{ event.category }}</p>
                <p><strong>Date:</strong> {{ event.date.strftime('%B %d, %Y at %I:%M %p') }}</p>
                <p><strong>Venue:</strong> {{ event.venue }}</p>
                <p><strong>Price:</strong> ${{ "%.2f"|format(event.price) }}</p>
            </div>
            {% if 'user_id' in session %}
                {% if is_booked %}
                    <button class="btn" disabled>Already Booked</button>
                {% else %}
                    <form action="/api/bookings" method="POST">
                        <input type="hidden" name="csrf_token" value="{{ csrf_token() }}"/>
                        <input type="hidden" name="event_id" value="{{ event.id }}">
                        <button type="submit" class="btn">Book Now</button>
                    </form>
                {% endif %}
            {% else %}
                <p><a href="/login" class="btn">Login to Book</a></p>
            {% endif %}
        </div>
    </div>
</div>
{% endblock %}

Overwriting templates/event.html


In [5]:
%%writefile templates/congratulations.html
{% extends "base.html" %}
{% block content %}
<div class="container">
    <div class="congratulations">
        <h1>🎉 Congratulations! 🎉</h1>
        <p>Your event has been successfully booked!</p>
        <a href="/" class="btn">Back to Home</a>
    </div>
</div>
{% endblock %}

Overwriting templates/congratulations.html


In [6]:
%%writefile templates/admin.html
{% extends "base.html" %}
{% block content %}
<div class="container">
    <h1>Admin Panel</h1>
    <meta name="csrf-token" content="{{ csrf_token() }}">

    <div class="admin-sections">
        <!-- Events Section -->
        <section class="admin-section">
            <h2>Event Management</h2>
            <div class="admin-actions">
                <button id="create-event-btn" class="btn btn-primary">Create New Event</button>
            </div>
            <div class="table-container">
                <table class="admin-table" id="events-table">
                    <thead>
                        <tr>
                            <th>Name</th>
                            <th>Date</th>
                            <th>Venue</th>
                            <th>Price</th>
                            <th>Actions</th>
                        </tr>
                    </thead>
                    <tbody>
                        {% for event in events %}
                        <tr data-id="{{ event.id }}">
                            <td>{{ event.name }}</td>
                            <td>{{ event.date.strftime('%B %d, %Y at %I:%M %p') }}</td>
                            <td>{{ event.venue }}</td>
                            <td>${{ "%.2f"|format(event.price) }}</td>
                            <td class="actions">
                                <button class="btn-edit btn-warning" data-id="{{ event.id }}">Edit</button>
                                <button class="btn-delete btn-danger" data-id="{{ event.id }}" onclick="deleteEvent({{ event.id }})">Delete</button>
                            </td>
                        </tr>
                        {% endfor %}
                    </tbody>
                </table>
            </div>
        </section>

        <!-- User Management Section -->
        <section class="admin-section">
            <h2>User Management</h2>
            <div class="admin-actions">
                <button id="create-user-btn" class="btn btn-primary">Create New User</button>
            </div>
            <div class="table-container">
                <table class="admin-table" id="users-table">
                    <thead>
                        <tr>
                            <th>Username</th>
                            <th>Email</th>
                            <th>Role</th>
                            <th>Booked Events</th>
                            <th>Actions</th>
                        </tr>
                    </thead>
                    <tbody>
                        {% for user in users %}
                        <tr data-id="{{ user.id }}">
                            <td>{{ user.username }}</td>
                            <td>{{ user.email }}</td>
                            <td>{{ user.role }}</td>
                            <td>
                                {% for booking in user.bookings %}
                                {{ booking.event.name }}{% if not loop.last %}, {% endif %}
                                {% endfor %}
                            </td>
                            <td class="actions">
                                <button class="btn-edit-user btn-warning" data-id="{{ user.id }}">Edit
                                 <button class="btn-delete-user btn-danger" data-id="{{ user.id }}" onclick="deleteUser({{ user.id }})">Delete</button>
                            </td>
                        </tr>
                        {% endfor %}
                    </tbody>
                </table>
            </div>
        </section>
    </div>

    <!-- Event Modal -->
    <div id="event-modal" class="modal">
        <div class="modal-content draggable">
            <div class="modal-header">
                <h2 id="modal-title">Create Event</h2>
                <span class="close">&times;</span>
            </div>
            <div class="modal-body scrollable">
                <form id="event-form" method="POST">
                    <input type="hidden" name="csrf_token" value="{{ csrf_token() }}"/>
                    <input type="hidden" id="event-id" name="event_id">
                    <input type="hidden" name="_method" id="event-method" value="POST">
                    <div class="form-group">
                        <label for="name">Event Name</label>
                        <input type="text" id="name" name="name" required>
                    </div>
                    <div class="form-group">
                        <label for="description">Description</label>
                        <textarea id="description" name="description" required></textarea>
                    </div>
                    <div class="form-group">
                        <label for="category">Category</label>
                        <input type="text" id="category" name="category">
                    </div>
                    <div class="form-group">
                        <label for="date">Date</label>
                        <input type="datetime-local" id="date" name="date" required>
                    </div>
                    <div class="form-group">
                        <label for="venue">Venue</label>
                        <input type="text" id="venue" name="venue" required>
                    </div>
                    <div class="form-group">
                        <label for="price">Price</label>
                        <input type="number" id="price" name="price" step="0.01" required>
                    </div>
                    <div class="form-group">
                        <label for="image_url">Image URL</label>
                        <input type="text" id="image_url" name="image_url">
                        <div class="image-preview-container">
                            <img id="image-preview" src="" alt="Image Preview" class="image-preview">
                        </div>
                    </div>
                </form>
            </div>
            <div class="modal-footer">
                <button type="submit" form="event-form" class="btn btn-primary">Save</button>
                <button type="button" class="btn btn-secondary close-modal">Cancel</button>
            </div>
        </div>
    </div>

    <!-- User Modal -->
    <div id="user-modal" class="modal">
        <div class="modal-content draggable">
            <div class="modal-header">
                <h2 id="user-modal-title">Create User</h2>
                <span class="close">&times;</span>
            </div>
            <div class="modal-body scrollable">
                <form id="user-form" method="POST">
                    <input type="hidden" name="csrf_token" value="{{ csrf_token() }}"/>
                    <input type="hidden" id="user-id" name="user_id">
                    <input type="hidden" name="_method" id="user-method" value="POST">
                    <div class="form-group">
                        <label for="username">Username</label>
                        <input type="text" id="username" name="username" required>
                    </div>
                    <div class="form-group">
                        <label for="email">Email</label>
                        <input type="email" id="email" name="email" required>
                    </div>
                    <div class="form-group">
                        <label for="password">Password</label>
                        <input type="password" id="password" name="password" required>
                    </div>
                    <div class="form-group">
                        <label for="role">Role</label>
                        <select id="role" name="role" required>
                            <option value="user">User</option>
                            <option value="admin">Admin</option>
                        </select>
                    </div>
                </form>
            </div>
            <div class="modal-footer">
                <button type="submit" form="user-form" class="btn btn-primary">Save</button>
                <button type="button" class="btn btn-secondary close-modal">Cancel</button>
            </div>
        </div>
    </div>
</div>

<style>
  .spinner-border {
    display: inline-block;
    width: 1rem;
    height: 1rem;
    vertical-align: text-bottom;
    border: 0.15em solid currentColor;
    border-right-color: transparent;
    border-radius: 50%;
    animation: spinner-border 0.75s linear infinite;
}

@keyframes spinner-border {
    to { transform: rotate(360deg); }
}
    .modal {
        display: none;
        position: fixed;
        z-index: 1000;
        left: 0;
        top: 0;
        width: 100%;
        height: 100%;
        background-color: rgba(0,0,0,0.4);
    }

    .modal-content {
        background-color: #fefefe;
        margin: 5% auto;
        padding: 20px;
        border: 1px solid #888;
        width: 80%;
        max-width: 600px;
        max-height: 80vh;
        overflow-y: auto;
        position: relative;
    }

    .draggable {
        cursor: move;
    }

    .close {
        color: #aaa;
        float: right;
        font-size: 28px;
        font-weight: bold;
        cursor: pointer;
    }

    .close:hover {
        color: black;
    }

    .btn {
        padding: 8px 16px;
        border: none;
        border-radius: 4px;
        cursor: pointer;
        margin-right: 5px;
    }

    .btn-primary {
        background-color: #007bff;
        color: white;
    }

    .btn-secondary {
        background-color: #6c757d;
        color: white;
    }

    .btn-danger {
        background-color: #dc3545;
        color: white;
    }

    .btn-warning {
        background-color: #ffc107;
        color: black;
    }

    .actions {
        display: flex;
        gap: 5px;
    }

    .form-group {
        margin-bottom: 15px;
    }

    label {
        display: block;
        margin-bottom: 5px;
    }

    input[type="text"],
    input[type="email"],
    input[type="password"],
    input[type="number"],
    input[type="datetime-local"],
    select,
    textarea {
        width: 100%;
        padding: 8px;
        border: 1px solid #ddd;
        border-radius: 4px;
    }

    .image-preview {
        max-width: 100%;
        max-height: 200px;
        margin-top: 10px;
        display: none;
    }
</style>
<script>
document.addEventListener('DOMContentLoaded', function() {
    // Get DOM elements
    const eventModal = document.getElementById('event-modal');
    const userModal = document.getElementById('user-modal');
    const eventForm = document.getElementById('event-form');
    const userForm = document.getElementById('user-form');
    const createEventBtn = document.getElementById('create-event-btn');
    const createUserBtn = document.getElementById('create-user-btn');

    // Get CSRF token
    function getCSRFToken() {
        return document.querySelector('meta[name="csrf-token"]').content;
    }

    // Make modals draggable
    function makeDraggable(modal) {
        const header = modal.querySelector('.modal-header');
        let isDragging = false;
        let offsetX, offsetY;

        header.addEventListener('mousedown', (e) => {
            isDragging = true;
            offsetX = e.clientX - modal.getBoundingClientRect().left;
            offsetY = e.clientY - modal.getBoundingClientRect().top;
            modal.style.cursor = 'grabbing';
        });

        document.addEventListener('mousemove', (e) => {
            if (!isDragging) return;

            const x = e.clientX - offsetX;
            const y = e.clientY - offsetY;

            const maxX = window.innerWidth - modal.offsetWidth;
            const maxY = window.innerHeight - modal.offsetHeight;

            modal.style.left = `${Math.min(Math.max(0, x), maxX)}px`;
            modal.style.top = `${Math.min(Math.max(0, y), maxY)}px`;
        });

        document.addEventListener('mouseup', () => {
            isDragging = false;
            modal.style.cursor = 'move';
        });
    }

    makeDraggable(eventModal.querySelector('.modal-content'));
    makeDraggable(userModal.querySelector('.modal-content'));

    // Modal open/close functions
    function openModal(modal) {
        modal.style.display = 'block';
    }

    function closeModals() {
        eventModal.style.display = 'none';
        userModal.style.display = 'none';
    }

    // Close modals when clicking X or cancel button
    document.querySelectorAll('.close, .close-modal').forEach(btn => {
        btn.addEventListener('click', closeModals);
    });

    // Close modals when clicking outside
    window.addEventListener('click', (e) => {
        if (e.target === eventModal || e.target === userModal) {
            closeModals();
        }
    });

    // Enhanced fetch with JSON handling
    async function fetchJSON(url, options = {}) {
        try {
            const response = await fetch(url, {
                ...options,
                headers: {
                    'Content-Type': 'application/json',
                    'Accept': 'application/json',
                    'X-CSRFToken': getCSRFToken(),
                    ...options.headers
                }
            });

            if (!response.ok) {
                const errorData = await response.json().catch(() => ({}));
                throw new Error(errorData.message || `HTTP error! status: ${response.status}`);
            }

            return await response.json();
        } catch (error) {
            console.error('Fetch error:', error);
            throw error;
        }
    }

    // Create Event Button
    createEventBtn.addEventListener('click', () => {
        document.getElementById('modal-title').textContent = 'Create Event';
        document.getElementById('event-id').value = '';
        document.getElementById('event-method').value = 'POST';
        eventForm.reset();
        eventForm.action = '/api/events';
        openModal(eventModal);
    });

    // Edit Event Button
    document.addEventListener('click', (e) => {
        if (e.target.classList.contains('btn-edit')) {
            const eventId = e.target.getAttribute('data-id');
            fetchJSON(`/api/events/${eventId}`)
                .then(event => {
                    document.getElementById('modal-title').textContent = 'Edit Event';
                    document.getElementById('event-id').value = event.id;
                    document.getElementById('event-method').value = 'PUT';
                    document.getElementById('name').value = event.name;
                    document.getElementById('description').value = event.description;
                    document.getElementById('category').value = event.category || '';

                    const date = new Date(event.date);
                    const localDate = new Date(date.getTime() - (date.getTimezoneOffset() * 60000))
                        .toISOString()
                        .slice(0, 16);
                    document.getElementById('date').value = localDate;

                    document.getElementById('venue').value = event.venue;
                    document.getElementById('price').value = event.price;
                    document.getElementById('image_url').value = event.image_url || '';

                    const preview = document.getElementById('image-preview');
                    if (event.image_url) {
                        preview.src = event.image_url;
                        preview.style.display = 'block';
                    } else {
                        preview.style.display = 'none';
                    }

                    eventForm.action = `/api/events/${event.id}`;
                    openModal(eventModal);
                })
                .catch(error => {
                    console.error('Error:', error);
                    showToast('Failed to load event data: ' + error.message, 'error');
                });
        }
    });

    // Delete Event Function
    async function deleteEvent(eventId, btnElement) {
        if (!confirm('Are you sure you want to delete this event?')) return;

        btnElement.disabled = true;
        btnElement.innerHTML = '<span class="spinner-border spinner-border-sm" role="status"></span> Deleting...';

        try {
            const response = await fetch(`/api/events/${eventId}`, {
                method: 'DELETE',
                headers: {
                    'Content-Type': 'application/json',
                    'X-CSRFToken': getCSRFToken()
                }
            });

            const data = await response.json();

            if (!response.ok || !data.success) {
                if (data.has_bookings) {
                    if (confirm('This event has bookings. Do you want to delete it anyway?')) {
                        const forceResponse = await fetch(`/api/events/${eventId}?force=true`, {
                            method: 'DELETE',
                            headers: {
                                'Content-Type': 'application/json',
                                'X-CSRFToken': getCSRFToken()
                            }
                        });
                        const forceData = await forceResponse.json();
                        handleDeleteResponse(forceData, eventId, 'event');
                    }
                    return;
                }
                throw new Error(data.error || 'Failed to delete event');
            }

            handleDeleteResponse(data, eventId, 'event');
        } catch (error) {
            console.error('Error:', error);
            showToast('Error: ' + error.message, 'error');
        } finally {
            btnElement.disabled = false;
            btnElement.textContent = 'Delete';
        }
    }

    // Create User Button
    createUserBtn.addEventListener('click', () => {
        document.getElementById('user-modal-title').textContent = 'Create User';
        document.getElementById('user-id').value = '';
        document.getElementById('user-method').value = 'POST';
        userForm.reset();
        userForm.action = '/api/users';
        document.getElementById('password').required = true;
        openModal(userModal);
    });

    // Edit User Button
    document.addEventListener('click', (e) => {
        if (e.target.classList.contains('btn-edit-user')) {
            const userId = e.target.getAttribute('data-id');
            fetchJSON(`/api/users/${userId}`)
                .then(user => {
                    document.getElementById('user-modal-title').textContent = 'Edit User';
                    document.getElementById('user-id').value = user.id;
                    document.getElementById('user-method').value = 'PUT';
                    document.getElementById('username').value = user.username;
                    document.getElementById('email').value = user.email;
                    document.getElementById('role').value = user.role;
                    document.getElementById('password').required = false;

                    userForm.action = `/api/users/${user.id}`;
                    openModal(userModal);
                })
                .catch(error => {
                    console.error('Error:', error);
                    showToast('Failed to load user data: ' + error.message, 'error');
                });
        }
    });

    // Delete User Function
    async function deleteUser(userId, btnElement) {
        if (!confirm('Are you sure you want to delete this user?')) return;

        btnElement.disabled = true;
        btnElement.innerHTML = '<span class="spinner-border spinner-border-sm" role="status"></span> Deleting...';

        try {
            const response = await fetch(`/api/users/${userId}`, {
                method: 'DELETE',
                headers: {
                    'Content-Type': 'application/json',
                    'X-CSRFToken': getCSRFToken()
                }
            });

            const data = await response.json();

            if (!response.ok || !data.success) {
                if (data.is_self) {
                    showToast('You cannot delete your own account!', 'error');
                    return;
                }
                throw new Error(data.error || 'Failed to delete user');
            }

            handleDeleteResponse(data, userId, 'user');
        } catch (error) {
            console.error('Error:', error);
            showToast('Error: ' + error.message, 'error');
        } finally {
            btnElement.disabled = false;
            btnElement.textContent = 'Delete';
        }
    }

    // Handle successful deletion
    function handleDeleteResponse(data, id, type) {
        if (data.success) {
            const row = document.querySelector(`tr[data-id="${id}"]`);
            if (row) {
                row.remove();
            }
            showToast(`${type.charAt(0).toUpperCase() + type.slice(1)} deleted successfully`, 'success');
        } else {
            showToast(data.error || `Failed to delete ${type}`, 'error');
        }
    }

    // Show toast notification
    function showToast(message, type) {
        const toast = document.createElement('div');
        toast.className = `toast align-items-center text-white bg-${type} border-0 show`;
        toast.innerHTML = `
            <div class="d-flex">
                <div class="toast-body">${message}</div>
                <button type="button" class="btn-close btn-close-white me-2 m-auto" data-bs-dismiss="toast"></button>
            </div>
        `;
        document.getElementById('toast-container').appendChild(toast);
        setTimeout(() => toast.remove(), 5000);
    }

    // Handle form submissions
    function handleFormSubmit(form, successCallback) {
        form.addEventListener('submit', async (e) => {
            e.preventDefault();
            const formData = new FormData(form);
            const action = form.action;
            const method = form.querySelector('[name="_method"]').value;

            const jsonData = {};
            formData.forEach((value, key) => {
                jsonData[key] = value;
            });

            try {
                const data = await fetchJSON(action, {
                    method: method,
                    body: JSON.stringify(jsonData)
                });

                if (data.success) {
                    if (successCallback) successCallback(data);
                    location.reload();
                } else {
                    throw new Error(data.error || 'Unknown error');
                }
            } catch (error) {
                console.error('Error:', error);
                showToast('Error: ' + error.message, 'error');
            }
        });
    }

    handleFormSubmit(eventForm);
    handleFormSubmit(userForm);

    // Image preview functionality
    const imageUrlInput = document.getElementById('image_url');
    if (imageUrlInput) {
        imageUrlInput.addEventListener('input', () => {
            const preview = document.getElementById('image-preview');
            if (imageUrlInput.value) {
                preview.src = imageUrlInput.value;
                preview.style.display = 'block';
            } else {
                preview.style.display = 'none';
            }
        });
    }

    // Event delegation for delete buttons
    document.addEventListener('click', (e) => {
        if (e.target.classList.contains('btn-delete')) {
            deleteEvent(e.target.getAttribute('data-id'), e.target);
        }
        if (e.target.classList.contains('btn-delete-user')) {
            deleteUser(e.target.getAttribute('data-id'), e.target);
        }
    });
});
</script>
{% endblock %}

Overwriting templates/admin.html


In [7]:
%%writefile templates/login.html
{% extends "base.html" %}
{% block content %}
<div class="container">
    <div class="auth-form">
        <h1>Login</h1>
        {% with messages = get_flashed_messages(with_categories=true) %}
            {% if messages %}
                <div class="flash-messages">
                    {% for category, message in messages %}
                        <div class="flash {{ category }}">{{ message }}</div>
                    {% endfor %}
                </div>
            {% endif %}
        {% endwith %}
        <form action="/api/login" method="POST">
            <input type="hidden" name="csrf_token" value="{{ csrf_token() }}"/>
            <div class="form-group">
                <label for="username">Username</label>
                <input type="text" id="username" name="username" required>
            </div>
            <div class="form-group">
                <label for="password">Password</label>
                <input type="password" id="password" name="password" required>
            </div>
            <button type="submit" class="btn">Login</button>
        </form>
        <p>Don't have an account? <a href="/register">Register here</a></p>
    </div>
</div>
{% endblock %}

Overwriting templates/login.html


In [8]:
%%writefile templates/register.html
{% extends "base.html" %}

{% block content %}
<div class="container">
    <div class="auth-form">
        <h1>Register</h1>
        {% with messages = get_flashed_messages(with_categories=true) %}
            {% if messages %}
                <div class="flash-messages">
                    {% for category, message in messages %}
                        <div class="flash {{ category }}">{{ message }}</div>
                    {% endfor %}
                </div>
            {% endif %}
        {% endwith %}
        <form action="/api/register" method="POST" enctype="application/x-www-form-urlencoded">
            <input type="hidden" name="csrf_token" value="{{ csrf_token() }}"/>
            <div class="form-group">
                <label for="username">Username</label>
                <input type="text" id="username" name="username" required>
            </div>
            <div class="form-group">
                <label for="email">Email</label>
                <input type="email" id="email" name="email" required>
            </div>
            <div class="form-group">
                <label for="password">Password</label>
                <input type="password" id="password" name="password" required>
            </div>
            <button type="submit" class="btn">Register</button>
        </form>
        <p>Already have an account? <a href="/login">Login here</a></p>
    </div>
</div>
{% endblock %}

Overwriting templates/register.html


In [9]:
%%writefile static/css/styles.css
/* Base Styles */
:root {
    --primary-color: #3498db;
    --secondary-color: #2980b9;
    --dark-color: #2c3e50;
    --light-color: #ecf0f1;
    --success-color: #2ecc71;
    --danger-color: #e74c3c;
}

* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

body {
    background-color: #f5f5f5;
    color: #333;
    line-height: 1.6;
}

.container {
    max-width: 1200px;
    margin: 0 auto;
    padding: 0 20px;
}

/* Header Styles */
header {
    background-color: var(--dark-color);
    color: white;
    padding: 1rem 0;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
}

nav {
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 0 2rem;
}


.logo {
    font-size: 1.8rem;
    font-weight: bold;
    color: white;
}

.nav-links {
    display: flex;
    gap: 1.5rem;
    align-items: center;
}

.nav-links a {
    color: white;
    text-decoration: none;
    font-weight: 500;
    transition: color 0.3s;
}

.nav-links a:hover {
    color: var(--primary-color);
}

/* Button Styles */
.btn {
    display: inline-block;
    background-color: var(--primary-color);
    color: white;
    padding: 0.5rem 1rem;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    text-decoration: none;
    font-size: 1rem;
    transition: background-color 0.3s;
}

.btn:hover {
    background-color: var(--secondary-color);
}

.btn-link {
    background: none;
    border: none;
    color: white;
    cursor: pointer;
    font-size: 1rem;
    padding: 0;
    font-weight: 500;
    transition: color 0.3s;
}

.btn-link:hover {
    color: var(--primary-color);
    text-decoration: underline;
}

/* Form Styles */
.auth-form {
    max-width: 500px;
    margin: 2rem auto;
    padding: 2rem;
    background: white;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0,0,0,0.1);
}

.auth-form h1 {
    margin-bottom: 1.5rem;
    text-align: center;
    color: var(--dark-color);
}

.form-group {
    margin-bottom: 1.5rem;
}

.form-group label {
    display: block;
    margin-bottom: 0.5rem;
    font-weight: 500;
}

.form-group input {
    width: 100%;
    padding: 0.75rem;
    border: 1px solid #ddd;
    border-radius: 4px;
    font-size: 1rem;
}

/* Events Grid */
.events-grid {
    display: grid;
    grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
    gap: 2rem;
    margin: 2rem 0;
}

.event-card {
    background: white;
    border-radius: 8px;
    overflow: hidden;
    box-shadow: 0 0 10px rgba(0,0,0,0.1);
    transition: transform 0.3s;
}

.event-card:hover {
    transform: translateY(-5px);
}

.event-card img {
    width: 100%;
    height: 200px;
    object-fit: cover;
}

.event-card h3 {
    padding: 1rem 1rem 0;
    color: var(--dark-color);
}

.event-card p {
    padding: 0 1rem;
    margin: 0.5rem 0;
    color: #666;
}

.event-card .btn {
    display: block;
    margin: 1rem;
    text-align: center;
}

/* Footer */
footer {
    background-color: var(--dark-color);
    color: white;
    text-align: center;
    padding: 1.5rem 0;
    margin-top: 2rem;
}

/* Flash Messages */
.flash-messages {
    margin: 1rem auto;
    max-width: 1200px;
    padding: 0 2rem;
}

.flash {
    padding: 1rem;
    margin-bottom: 1rem;
    border-radius: 4px;
}

.flash.success {
    background-color: #d4edda;
    color: #155724;
    border: 1px solid #c3e6cb;
}

.flash.error {
    background-color: #f8d7da;
    color: #721c24;
    border: 1px solid #f5c6cb;
}

/* Event Details */
.event-details {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 2rem;
    margin: 2rem 0;
    background: white;
    padding: 2rem;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0,0,0,0.1);
}

.event-image {
    width: 100%;
    height: auto;
    border-radius: 8px;
}

.event-info h1 {
    color: var(--dark-color);
    margin-bottom: 1rem;
}

.event-meta {
    margin: 1.5rem 0;
}

.event-meta p {
    margin: 0.5rem 0;
}

/* Admin Styles */
.admin-table {
    width: 100%;
    border-collapse: collapse;
    margin: 2rem 0;
    background: white;
    box-shadow: 0 0 10px rgba(0,0,0,0.1);
}

.admin-table th, .admin-table td {
    padding: 1rem;
    text-align: left;
    border-bottom: 1px solid #ddd;
}

.admin-table th {
    background-color: var(--dark-color);
    color: white;
}

.btn-edit, .btn-delete {
    padding: 0.5rem 1rem;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    margin-right: 0.5rem;
}

.btn-edit {
    background-color: #f39c12;
    color: white;
}

.btn-delete {
    background-color: var(--danger-color);
    color: white;
}

/* Modal Styles */
.modal {
    display: none;
    position: fixed;
    z-index: 1000;
    left: 0;
    top: 0;
    width: 100%;
    height: 100%;
    background-color: rgba(0,0,0,0.5);
}

.modal-content {
    background-color: white;
    margin: 5% auto;
    padding: 2rem;
    border-radius: 8px;
    width: 80%;
    max-width: 600px;
    position: relative;
}

.close {
    position: absolute;
    right: 1rem;
    top: 1rem;
    font-size: 1.5rem;
    cursor: pointer;
}

/* Congratulations Page */
.congratulations {
    text-align: center;
    padding: 4rem 0;
}

.congratulations h1 {
    font-size: 3rem;
    margin-bottom: 1.5rem;
    color: var(--primary-color);
}

.congratulations p {
    font-size: 1.2rem;
    margin-bottom: 2rem;
}

/* Booked label */
.booked-label {
    display: inline-block;
    background-color: var(--success-color);
    color: white;
    padding: 0.5rem 1rem;
    border-radius: 4px;
    margin: 1rem;
    font-weight: bold;
}

/* Form improvements */
form {
    margin: 1rem 0;
}

input, textarea, select {
    width: 100%;
    padding: 0.75rem;
    margin: 0.5rem 0;
    border: 1px solid #ddd;
    border-radius: 4px;
    font-size: 1rem;
}

textarea {
    min-height: 100px;
    resize: vertical;
}

/* Add to your existing CSS */
.modal {
    display: none;
    position: fixed;
    z-index: 1000;
    left: 0;
    top: 0;
    width: 100%;
    height: 100%;
    background-color: rgba(0,0,0,0.5);
}

.modal-content {
    background-color: white;
    margin: 5% auto;
    padding: 2rem;
    border-radius: 8px;
    width: 80%;
    max-width: 600px;
    position: absolute;
    left: 50%;
    top: 50%;
    transform: translate(-50%, -50%);
    cursor: grab;
}

.modal-content:active {
    cursor: grabbing;
}

.admin-sections {
    display: grid;
    grid-template-columns: 1fr;
    gap: 2rem;
}

@media (min-width: 992px) {
    .admin-sections {
        grid-template-columns: 1fr 1fr;
    }
}

.admin-section {
    background: white;
    padding: 1.5rem;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0,0,0,0.1);
}

Overwriting static/css/styles.css


In [10]:
%%writefile static/js/main.js
// Handle logout confirmation
document.addEventListener('DOMContentLoaded', function() {
    // Confirm before deleting events
    document.querySelectorAll('form[action*="/delete"]').forEach(form => {
        form.addEventListener('submit', function(e) {
            if (!confirm('Are you sure you want to delete this event?')) {
                e.preventDefault();
            }
        });
    });
});

Overwriting static/js/main.js




---

* Needed Libraries :
---


---





In [11]:
!pip install flask-sqlalchemy
!pip install flask werkzeug pyngrok
!pip install Flask-Babel
!pip install flask-wtf
!ngrok authtoken 2tJcCOcKawyNHUrD3fx8ZGX7mqf_UpyxfMvX1tmYtgH2vSdR

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml




---
* App using Flask :
---
---



In [ ]:
from flask import Flask, request, jsonify, session, redirect, url_for, render_template, flash
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash
import os
from datetime import datetime
from functools import wraps
from pyngrok import ngrok ,  conf
from flask_wtf.csrf import CSRFProtect
import atexit
from flask_babel import Babel, _



# First create the Flask app instance
app = Flask(__name__)

# Then configure it
app.secret_key = os.urandom(24)  # Or use a fixed string for development
app.config['SECRET_KEY'] = 'os.urandom(24) '  # For Flask-WTF
app.config['WTF_CSRF_ENABLED'] = True
app.config['WTF_CSRF_SECRET_KEY'] = 'another-secret-key-here'  # Different from SECRET_KEY
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///events.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['BABEL_DEFAULT_LOCALE'] = 'en'
babel = Babel(app)

# Then initialize extensions
csrf = CSRFProtect(app)
db = SQLAlchemy(app)


# Models
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password = db.Column(db.String(200), nullable=False)
    role = db.Column(db.String(20), default='user')
    bookings = db.relationship('Booking', backref='user', lazy=True)

class Event(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    description = db.Column(db.Text, nullable=False)
    category = db.Column(db.String(50))
    date = db.Column(db.DateTime, nullable=False)
    venue = db.Column(db.String(100), nullable=False)
    price = db.Column(db.Float, nullable=False)
    image_url = db.Column(db.String(200))
    bookings = db.relationship('Booking', backref='event', lazy=True)

class Booking(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    event_id = db.Column(db.Integer, db.ForeignKey('event.id'), nullable=False)
    booking_date = db.Column(db.DateTime, default=datetime.utcnow)
    tickets = db.Column(db.Integer, default=1)

# Create tables
with app.app_context():
    db.create_all()
    # Create admin user if not exists
    if not User.query.filter_by(username='admin').first():
        admin = User(
            username='admin',
            email='admin@example.com',
            password=generate_password_hash('admin123'),
            role='admin'
        )
        db.session.add(admin)
        db.session.commit()

def create_sample_events():
    if not Event.query.first():  # Only create if no events exist
        sample_events = [
            Event(
                name="Tech Conference 2025",
                description="Annual technology conference featuring the latest innovations",
                category="Technology",
                date=datetime(2025, 6, 15, 9, 0),
                venue="Convention Center",
                price=199.99,
                image_url="https://images.unsplash.com/photo-1505373877841-8d25f7d46678"
            ),
            Event(
                name="Jazz Night",
                description="An evening of smooth jazz with world-class musicians",
                category="Music",
                date=datetime(2025, 7, 20, 19, 30),
                venue="City Hall Auditorium",
                price=49.99,
                image_url="https://images.unsplash.com/photo-1493225457124-a3eb161ffa5f"
            ),
            Event(
                name="Food Festival",
                description="Taste the best local and international cuisines",
                category="Food",
                date=datetime(2025, 8, 5, 11, 0),
                venue="Central Park",
                price=25.00,
                image_url="https://images.unsplash.com/photo-1504674900247-0877df9cc836"
            )
        ]
        db.session.add_all(sample_events)
        db.session.commit()



# Context processor to make current_user available in all templates
@app.context_processor
def inject_user():
    if 'user_id' in session:
        user = User.query.get(session['user_id'])
        return {'current_user': user}
    return {'current_user': None}

# Decorators
def login_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if 'user_id' not in session:
            flash('Please log in to access this page', 'error')
            return redirect(url_for('login_page'))
        return f(*args, **kwargs)
    return decorated_function

def admin_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if 'user_id' not in session:
            flash('Please log in to access this page', 'error')
            return redirect(url_for('login_page'))
        user = User.query.get(session['user_id'])
        if user.role != 'admin':
            flash('Admin access required', 'error')
            return redirect(url_for('home'))
        return f(*args, **kwargs)
    return decorated_function

# Auth Routes
@app.route('/api/register', methods=['POST'])
def register():
    username = request.form.get('username')
    email = request.form.get('email')
    password = request.form.get('password')

    if not all([username, email, password]):
        flash('All fields are required', 'error')
        return redirect(url_for('register_page'))

    if User.query.filter_by(username=username).first():
        flash('Username already exists', 'error')
        return redirect(url_for('register_page'))

    if User.query.filter_by(email=email).first():
        flash('Email already exists', 'error')
        return redirect(url_for('register_page'))

    user = User(
        username=username,
        email=email,
        password=generate_password_hash(password),
        role='user'
    )
    db.session.add(user)
    db.session.commit()

    flash('Registration successful! Please login.', 'success')
    return redirect(url_for('login_page'))

@app.route('/api/login', methods=['POST'])
def login():
    username = request.form.get('username')
    password = request.form.get('password')
    user = User.query.filter_by(username=username).first()

    if not user or not check_password_hash(user.password, password):
        flash('Invalid credentials', 'error')
        return redirect(url_for('login_page'))

    session['user_id'] = user.id
    flash('Logged in successfully', 'success')
    return redirect(url_for('home'))

@app.route('/api/logout', methods=['POST'])
@login_required
def logout():
    session.pop('user_id', None)
    flash('Logged out successfully', 'success')
    return redirect(url_for('home'))

# Frontend Routes

@app.route('/')
def home():
    try:
        events = Event.query.filter(Event.date >= datetime.now()).order_by(Event.date.asc()).all()
        booked_events = []
        if 'user_id' in session:
            booked_events = [b.event_id for b in Booking.query.filter_by(user_id=session['user_id']).all()]
        return render_template('index.html', events=events, booked_events=booked_events)
    except Exception as e:
        app.logger.error(f"Error in home route: {str(e)}")
        flash('An error occurred loading events', 'error')
        return render_template('index.html', events=[], booked_events=[])


@app.route('/event/<int:event_id>')
def event_details(event_id):
    event = Event.query.get_or_404(event_id)
    is_booked = False
    if 'user_id' in session:
        is_booked = Booking.query.filter_by(
            user_id=session['user_id'],
            event_id=event_id
        ).first() is not None
    return render_template('event.html', event=event, is_booked=is_booked)

@app.route('/congratulations')
def congratulations():
    return render_template('congratulations.html')



@app.route('/login', methods=['GET'])
def login_page():
    return render_template('login.html')

@app.route('/register', methods=['GET'])
def register_page():
    return render_template('register.html')

@app.route('/api/events', methods=['GET', 'POST'])
@admin_required
def manage_events():
    if request.method == 'POST':
        try:
            # Handle both form data and JSON requests
            data = request.get_json() if request.is_json else request.form
            event = Event(
                name=data.get('name'),
                description=data.get('description'),
                category=data.get('category'),
                date=datetime.strptime(data.get('date'), '%Y-%m-%dT%H:%M'),
                venue=data.get('venue'),
                price=float(data.get('price')),
                image_url=data.get('image_url')
            )
            db.session.add(event)
            db.session.commit()
            if request.is_json:
                return jsonify({'success': True, 'id': event.id})
            flash('Event created successfully', 'success')
            return redirect(url_for('admin_panel'))
        except Exception as e:
            db.session.rollback()
            if request.is_json:
                return jsonify({'success': False, 'error': str(e)}), 400
            flash(f'Error creating event: {str(e)}', 'error')
            return redirect(url_for('admin_panel'))
    elif request.method == 'GET':
        events = Event.query.all()
        return jsonify([{
            'id': e.id,
            'name': e.name,
            'date': e.date.isoformat(),
            'venue': e.venue,
            'price': e.price
        } for e in events])


@app.route('/api/users', methods=['GET', 'POST'])
@admin_required
def manage_users():
    if request.method == 'POST':
        try:
            data = request.get_json() if request.is_json else request.form
            if not all([data.get('username'), data.get('email'), data.get('password')]):
                raise ValueError('All fields are required')
            if User.query.filter_by(username=data.get('username')).first():
                raise ValueError('Username already exists')
            if User.query.filter_by(email=data.get('email')).first():
                raise ValueError('Email already exists')
            user = User(
                username=data.get('username'),
                email=data.get('email'),
                password=generate_password_hash(data.get('password')),
                role=data.get('role', 'user')
            )
            db.session.add(user)
            db.session.commit()
            if request.is_json:
                return jsonify({'success': True, 'id': user.id})
            flash('User created successfully', 'success')
            return redirect(url_for('admin_panel'))
        except Exception as e:
            db.session.rollback()
            if request.is_json:
                return jsonify({'success': False, 'error': str(e)}), 400
            flash(f'Error creating user: {str(e)}', 'error')
            return redirect(url_for('admin_panel'))
    elif request.method == 'GET':
        users = User.query.all()
        return jsonify([{
            'id': u.id,
            'username': u.username,
            'email': u.email,
            'role': u.role
        } for u in users])











@app.route('/api/events/<int:event_id>', methods=['GET', 'PUT', 'DELETE'])
@admin_required
def manage_event(event_id):
    event = Event.query.get_or_404(event_id)
    if request.method == 'GET':
        return jsonify({
            'id': event.id,
            'name': event.name,
            'description': event.description,
            'category': event.category,
            'date': event.date.isoformat(),
            'venue': event.venue,
            'price': event.price,
            'image_url': event.image_url
        })
    elif request.method == 'PUT':
        try:
            data = request.get_json() if request.is_json else request.form
            event.name = data.get('name', event.name)
            event.description = data.get('description', event.description)
            event.category = data.get('category', event.category)
            if 'date' in data:
                event.date = datetime.strptime(data.get('date'), '%Y-%m-%dT%H:%M')
            event.venue = data.get('venue', event.venue)
            if 'price' in data:
                event.price = float(data.get('price'))
            event.image_url = data.get('image_url', event.image_url)
            db.session.commit()
            return jsonify({'success': True})

        except Exception as e:
            db.session.rollback()
            return jsonify({'success': False, 'error': str(e)}), 400
    elif request.method == 'DELETE':
        try:
            # First check if there are any bookings
            has_bookings = Booking.query.filter_by(event_id=event_id).first() is not None
            if has_bookings:
                # Option 1: Delete all bookings first (uncomment if you want this)
                Booking.query.filter_by(event_id=event_id).delete()
                # Option 2: Return error (currently implemented)
                # return jsonify({
                #     'success': False,
                #     'error': 'Event has bookings and cannot be deleted',
                #     'has_bookings': True
                # }), 400
            db.session.delete(event)
            db.session.commit()
            return jsonify({
                'success': True,
                'message': 'Event deleted successfully',
                'deleted_id': event_id
            })
        except Exception as e:
            db.session.rollback()
            return jsonify({
                'success': False,
                'error': str(e),
                'has_bookings': False
            }), 400





@app.route('/api/users/<int:user_id>', methods=['GET', 'PUT', 'DELETE'])
@admin_required
def manage_user(user_id):
    user = User.query.get_or_404(user_id)
    if request.method == 'GET':
        return jsonify({
            'id': user.id,
            'username': user.username,
            'email': user.email,
            'role': user.role
        })
    elif request.method == 'PUT':
        try:
            data = request.get_json() if request.is_json else request.form
            user.username = data.get('username', user.username)
            user.email = data.get('email', user.email)
            user.role = data.get('role', user.role)
            if 'password' in data and data.get('password'):
                user.password = generate_password_hash(data.get('password'))
            db.session.commit()
            return jsonify({'success': True})
        except Exception as e:
            db.session.rollback()
            return jsonify({'success': False, 'error': str(e)}), 400
    elif request.method == 'DELETE':
        try:
            if user.id == session.get('user_id'):
                return jsonify({
                    'success': False,
                    'error': 'Cannot delete yourself',
                    'is_self': True
                }), 400
            # Delete user's bookings first
            Booking.query.filter_by(user_id=user_id).delete()
            db.session.delete(user)
            db.session.commit()
            return jsonify({
                'success': True,
                'message': 'User deleted successfully',
                'deleted_id': user_id
            })
        except Exception as e:
            db.session.rollback()
            return jsonify({
                'success': False,
                'error': str(e),
                'is_self': False
            }), 400


@app.route('/api/bookings', methods=['POST'])
@login_required
def create_booking():
    event_id = request.form.get('event_id')
    if not event_id:
        flash('Event ID is required', 'error')
        return redirect(url_for('home'))

    event = Event.query.get(event_id)
    if not event:
        flash('Event not found', 'error')
        return redirect(url_for('home'))

    # Check if already booked
    existing_booking = Booking.query.filter_by(
        user_id=session['user_id'],
        event_id=event_id
    ).first()

    if existing_booking:
        flash('You have already booked this event', 'error')
        return redirect(url_for('event_details', event_id=event_id))

    try:
        booking = Booking(
            user_id=session['user_id'],
            event_id=event_id
        )
        db.session.add(booking)
        db.session.commit()
        return redirect(url_for('congratulations'))
    except Exception as e:
        db.session.rollback()
        flash(f'Error creating booking: {str(e)}', 'error')
        return redirect(url_for('event_details', event_id=event_id))
#

# Update admin_panel route to include users
@app.route('/admin')
@admin_required
def admin_panel():
    events = Event.query.order_by(Event.date.asc()).all()
    users = User.query.order_by(User.username.asc()).all()
    return render_template('admin.html', events=events, users=users)




if __name__ == '__main__':
    with app.app_context():
        db.create_all()
        # Create admin user if not exists
        if not User.query.filter_by(username='admin').first():
            admin = User(
                username='admin',
                email='admin@example.com',
                password=generate_password_hash('admin123'),
                role='admin'
            )
            db.session.add(admin)
            db.session.commit()
        # Create sample events if none exist
        create_sample_events()
       # 4. Start tunnel with error handling
        public_url = ngrok.connect(5000)
        print(f" * Ngrok Tunnel: {public_url}")
        app.run(host='0.0.0.0', port=5000)

 * Ngrok Tunnel: NgrokTunnel: "https://adb0-34-60-60-241.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/May/2025 20:59:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/May/2025 20:59:20] "GET /static/css/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/May/2025 20:59:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/May/2025 20:59:24] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/May/2025 20:59:25] "GET /static/css/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [16/May/2025 20:59:32] "POST /api/login HTTP/1.1" 302 -
<ipython-input-12-45253bf59000>:113: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Sess



---

To access admin mode : u need to type : admin at user name , Password : admin123 , another comment u may be waiting a little bit for seeing any impcart or refresh tabs for any image thanks for this greate task , and i have choosen Jupter Notebook as its collecting whole at just once , I would like to deliver VSCode but I felt like its easy using Jupter NoteBook more :)

---

